# Librerias

In [1]:
import pandas as pd
import numpy as np

import logging


import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore")

import consultas as cst
from datetime import datetime, date, timedelta

pd.options.display.max_columns=False

In [2]:
#dia = int(str(datetime.now())[:10].replace('-', ''))
dia = 20230630
print(dia)
#mes = int(str(datetime.now())[:8].replace('-', ''))
mes = 202306
print(mes)
#today = date.today().strftime("%d/%m/%y")
today = '30/06/23'
print(today)

20230630
202306
30/06/23


# Conexión

In [3]:
#ruta = 'C:/Program Files/instantclient-basic-windows.x64-19.11.0.0.0dbru/instantclient_19_11'
ruta = 'C:/Users/womsrvdp/Documents/instantclient_19_11'
cx_Oracle.init_oracle_client(lib_dir = ruta)

NameError: name 'cx_Oracle' is not defined

In [4]:
dsn_tns = cx_Oracle.makedsn('10.41.87.18', '1521', service_name='DWHWOM')
conn = cx_Oracle.connect(user=r'ANALITICA_WOM', password='ANALI2023*5', dsn=dsn_tns)

NameError: name 'cx_Oracle' is not defined

# Insumos

## *Ending*

In [3]:
%%time
query=f"""
WITH 
ACTI AS (
    SELECT /*+PARALLEL(6)*/
        CAST(E.SUBSCRIBER_ID AS VARCHAR(20)) SUBSCRIBER_ID,
        E.PERIODO_PROCESO_CODIGO,
        E.FECHA_ALTA,
        E.ESTADO,
        E.MOVIMIENTO_NOMBRE,
        E.MOVIMIENTO_TIPO_NOMBRE,
        E.OPERADOR_PORTA_ORIGEN,
        UPPER (P.PLAN_NOMBRE) PLAN_NOMBRE,
        UPPER (P.PLAN_TIPO_COMERCIAL) PLAN_TIPO_COMERCIAL,
        P.PLAN_REVENUE,
        E.CICLO_FACTURACION,
        E.CONSULTOR_DK,
        DC.CONSULTOR_CANAL_NOMBRE,
        DC.CONSULTOR_SUBCANAL_NOMBRE,
        DC.CONSULTOR_CIUDAD_NOMBRE,
        DC.CONSULTOR_REGIONAL_NOMBRE,
        C.CLIENTE_GENERO,
        C.CLIENTE_NOMBRE,
        C.CLIENTE_EMAIL,
        C.CLIENTE_EDAD,
        C.CLIENTE_CIUDAD,
        C.CLIENTE_REGIONAL,
        C.CLIENTE_CODIGO_NK,
        CAST(E.MSISDN AS VARCHAR(20)) MSISDN,
        UPPER(G.GENERO_MODELO) GENERO,
        ACT.MOVIMIENTO_ACTIVACION,
        J.JINETE,
        TO_DATE('{today}','DD/MM/YY')- TO_DATE(E.FECHA_ALTA, 'DD/MM/YY') AS ANTIGUEDAD,
        ROW_NUMBER() OVER(PARTITION BY E.SUBSCRIBER_ID ORDER BY E.FECHA_ALTA DESC) ORDEN_A
    FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING E
    LEFT JOIN DWH_BODEGA_WOM.DIM_CLIENTES C ON E.CLIENTE_DK = C.CLIENTE_DK
    LEFT JOIN DWH_BODEGA_WOM.DIM_CONSULTORES DC ON E.CONSULTOR_DK = DC.CONSULTOR_DK
    LEFT JOIN DWH_BODEGA_WOM.DIM_PLANES P ON E.PLAN_COMERCIAL_DK = P.PLAN_COMERCIAL_DK
    LEFT JOIN GENERO_CLIENTES_SM G ON E.CLIENTE_DK = G.CLIENTE_DK
    LEFT JOIN(
            SELECT /*+PARALLEL(6)*/
                SUBSCRIBER_ID,
                MSISDN,
                MOVIMIENTO_TIPO_NOMBRE MOVIMIENTO_ACTIVACION,
                FECHA_ALTA
            FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING
                WHERE SERVICIO='Postpaid'
                AND MOVIMIENTO_NOMBRE ='ACTIVACION') ACT ON ((E.SUBSCRIBER_ID=ACT.SUBSCRIBER_ID AND E.FECHA_ALTA=ACT.FECHA_ALTA) 
                                                                OR (E.SUBSCRIBER_ID=ACT.SUBSCRIBER_ID AND E.MSISDN=ACT.MSISDN))
    LEFT JOIN (SELECT
                    DISTINCT CLIENTE_DK ,
                    1 JINETE
                FROM WOM_AA_CLIENTE_DK_JINETE) J ON E.CLIENTE_DK = J.CLIENTE_DK
        WHERE E.SERVICIO='Postpaid'
        ---AND E.ESTADO = 'Active'
        AND MOVIMIENTO_NOMBRE != 'DESCONEXION'
        AND E.PERIODO_PROCESO_CODIGO = {mes}
        AND (E.FECHA_BAJA IS NULL OR TO_DATE(E.FECHA_BAJA, 'DD/MM/YY') >= TO_DATE('{today}','DD/MM/YY'))
        
    ),

ACTIVADOS AS (
    SELECT /*+PARALLEL(6)*/
        A.SUBSCRIBER_ID,
        A.PERIODO_PROCESO_CODIGO,
        A.MSISDN,
        A.FECHA_ALTA,
        A.MOVIMIENTO_NOMBRE,
        A.ESTADO,
        A.MOVIMIENTO_TIPO_NOMBRE,
        A.OPERADOR_PORTA_ORIGEN,
        CASE 
            WHEN PLAN_TIPO_COMERCIAL = 'BROADBAND' THEN 'BROADBAND'
            WHEN PLAN_TIPO_COMERCIAL = 'B2B' THEN 'B2B'
            WHEN UPPER (PLAN_NOMBRE) LIKE '%PLAN XS%' THEN 'PLAN XS'
            WHEN UPPER (PLAN_NOMBRE) LIKE '%PLAN S%' THEN 'PLAN S'
            WHEN UPPER (PLAN_NOMBRE) LIKE '%AA 25_GB%' THEN 'PLAN S'
            WHEN UPPER (PLAN_NOMBRE) LIKE '%PLAN M%' THEN 'PLAN M'
            WHEN UPPER (PLAN_NOMBRE) LIKE '%ILI_GB%' THEN 'PLAN ILIMITADO'
            WHEN UPPER (PLAN_NOMBRE) LIKE '%ILIM_GB%' THEN 'PLAN ILIMITADO'
            WHEN UPPER (PLAN_NOMBRE) LIKE '%ILI GB%' THEN 'PLAN ILIMITADO'
            WHEN UPPER (PLAN_NOMBRE) LIKE '%ILIM GB%' THEN 'PLAN ILIMITADO'
            WHEN UPPER (PLAN_NOMBRE) LIKE '%ILIMI GB%' THEN 'PLAN ILIMITADO'
            WHEN UPPER (PLAN_NOMBRE) LIKE '%ILIMI_GB%' THEN 'PLAN ILIMITADO'
            WHEN UPPER (PLAN_NOMBRE) LIKE '%ILIMGB%' THEN 'PLAN ILIMITADO'
            WHEN UPPER (PLAN_NOMBRE) LIKE '%PLAN L%' THEN 'PLAN L'
            WHEN UPPER (PLAN_NOMBRE) LIKE '%PLAN XL%' THEN 'PLAN XL'
            WHEN UPPER (PLAN_NOMBRE) LIKE '%MÓVIL%' THEN 'INT MÓVIL'
            ELSE 'OTROS' END PLAN_NOMBRE,
        CASE 
            WHEN PLAN_TIPO_COMERCIAL = 'BROADBAND' THEN 'BROADBAND'
            WHEN PLAN_TIPO_COMERCIAL = 'MOBILE' THEN 'MOBILE'
            WHEN PLAN_TIPO_COMERCIAL = 'B2B' THEN 'B2B'
            ELSE 'SIN DEFINIR' END  PLAN_TIPO_COMERCIAL,
        A.PLAN_REVENUE,

        A.CICLO_FACTURACION,
        CASE 
            WHEN A.CICLO_FACTURACION LIKE '%01%' THEN '01'
            WHEN A.CICLO_FACTURACION LIKE '%05%' THEN '05'
            WHEN A.CICLO_FACTURACION LIKE '%10%' THEN '10'
            WHEN A.CICLO_FACTURACION LIKE '%15%' THEN '15'
            WHEN A.CICLO_FACTURACION LIKE '%20%' THEN '20'
            WHEN A.CICLO_FACTURACION LIKE '%25%' THEN '25'
            ELSE '00' END AS CICLO,
        A.CONSULTOR_DK,
        A.CLIENTE_GENERO,
        A.CLIENTE_EDAD,
        A.CLIENTE_CIUDAD,
        A.CLIENTE_REGIONAL,
        A.CLIENTE_NOMBRE,
        A.CLIENTE_EMAIL,
        A.CLIENTE_CODIGO_NK,
        CASE 
            WHEN CONSULTOR_CANAL_NOMBRE ='Tiendas' THEN 'Shops'
            WHEN CONSULTOR_CANAL_NOMBRE ='Fuerza De Venta Indirecta' THEN 'Dealers'
            WHEN CONSULTOR_CANAL_NOMBRE ='Outbound' THEN 'Outbound'
            WHEN CONSULTOR_CANAL_NOMBRE ='Inbound' THEN 'Inbound'
            WHEN CONSULTOR_CANAL_NOMBRE ='Kioscos' THEN 'Kiosks'
            WHEN CONSULTOR_CANAL_NOMBRE ='Ventas Online' THEN 'eCommerce'
            WHEN CONSULTOR_CANAL_NOMBRE = 'Tiendas Express' THEN 'Express Shops'
            WHEN CONSULTOR_CANAL_NOMBRE = 'Cadenas' THEN 'Retail'
        ELSE 'OTROS' END AS CONSULTOR_CANAL_NOMBRE,
        A.CONSULTOR_SUBCANAL_NOMBRE,
        A.CONSULTOR_CIUDAD_NOMBRE,
        A.CONSULTOR_REGIONAL_NOMBRE,
        A.ANTIGUEDAD,
        A.GENERO,
        A.MOVIMIENTO_ACTIVACION,
        A.JINETE
    FROM ACTI A
    WHERE A.ORDEN_A = 1
    ),
  
BD1 AS (
    SELECT  /*+PARALLEL(6)*/  
        ACT.SUBSCRIBER_ID,
        ACT.PERIODO_PROCESO_CODIGO,
        ACT.MSISDN,
        ACT.FECHA_ALTA,
        ACT.ANTIGUEDAD,
        ACT.MOVIMIENTO_NOMBRE,
        ACT.ESTADO,
        ACT.MOVIMIENTO_TIPO_NOMBRE,
        ACT.OPERADOR_PORTA_ORIGEN,
        ACT.PLAN_NOMBRE,
        ACT.PLAN_TIPO_COMERCIAL,
        ACT.PLAN_REVENUE,
        ACT.CICLO,
        ACT.CONSULTOR_DK,
        ACT.CLIENTE_CIUDAD,
        ACT.CLIENTE_REGIONAL,
        ACT.CONSULTOR_CANAL_NOMBRE,
        ACT.CONSULTOR_REGIONAL_NOMBRE,
        ACT.GENERO,
        ACT.MOVIMIENTO_ACTIVACION,
        
        ACT.JINETE
    FROM ACTIVADOS ACT
  )  
  
  SELECT /*+PARALLEL(6)*/  * FROM BD1
"""
ENDI=cst.consulta(query)
ENDI.shape

CPU times: total: 36.1 s
Wall time: 30min 50s


(1637813, 21)

In [6]:
ENDING=ENDI.copy()
ENDING['CONSULTOR_CANAL_NOMBRE']=np.where(ENDING['CONSULTOR_CANAL_NOMBRE'].isin(['-1','-2']),'NO IDENTIFICADO',ENDING['CONSULTOR_CANAL_NOMBRE'])
ENDING['OPERADOR_PORTA_ORIGEN']=np.where((ENDING.OPERADOR_PORTA_ORIGEN.isin(['COMCEL','TIGO','TELEFONICA'])|ENDING.OPERADOR_PORTA_ORIGEN.isna()),ENDING.OPERADOR_PORTA_ORIGEN,'OTROS')
ENDING['OPERADOR_PORTA_ORIGEN']=np.where(ENDING.OPERADOR_PORTA_ORIGEN.isna(),'LINEA NUEVA',ENDING.OPERADOR_PORTA_ORIGEN)

In [7]:
ENDING['REGION']=np.where(ENDING['CLIENTE_REGIONAL'].isin(['Bogotá D.C.','Bogota, D.C.']), 'Bogotá',
                          np.where(ENDING['CLIENTE_REGIONAL'].isin(['Antioquia','Risaralda','Caldas']) , 'Regional Antioquia',
                                   np.where(ENDING['CLIENTE_REGIONAL'].isin(['Atlántico','Bolívar','Magdalena','Cesar','Córdoba','Sucre','La Guajira']) ,'Regional Costa',
                                            np.where(ENDING['CLIENTE_REGIONAL'].isin(['Tolima','Risaralda','Huila','Caldas','Quindío']),'Regional Eje Cafetero',
                                                     np.where(ENDING['CLIENTE_REGIONAL'].isin(['Valle Del Cauca','Nariño','Cauca','Putumayo','Chocó']),'Regional Occidente',
                                                              np.where(ENDING['CLIENTE_REGIONAL'].isin(['Santander','Norte De Santander','Boyacá','Casanare','Arauca','Guaviare']),'Regional Oriente',
                                                                       np.where(ENDING['CLIENTE_REGIONAL'].isin(['Cundinamarca','Meta','Caquetá']),'Regional Centro',ENDING['CONSULTOR_REGIONAL_NOMBRE'])))))))

In [8]:
ENDING['REGION']=np.where(ENDING.REGION=='-2','IND',ENDING.REGION)

In [9]:
ENDING['MOVIMIENTO_ACTIVACION']=ENDING['MOVIMIENTO_ACTIVACION'].fillna('GROSS ADDS MIGRACION')
##GROSS ADDS MIGRACION

In [10]:
ENDING['MOVIMIENTO_ACTIVACION'].value_counts()

GROSS ADDS PORTACION    1094357
GROSS ADDS NUEVA         384731
GROSS ADDS MIGRACION     101516
Name: MOVIMIENTO_ACTIVACION, dtype: int64

In [11]:
ENDING[ENDING['MOVIMIENTO_ACTIVACION']=='REV']

,SUBSCRIBER_ID,PERIODO_PROCESO_CODIGO,MSISDN,FECHA_ALTA,ANTIGUEDAD,MOVIMIENTO_NOMBRE,ESTADO,MOVIMIENTO_TIPO_NOMBRE,OPERADOR_PORTA_ORIGEN,PLAN_NOMBRE,PLAN_TIPO_COMERCIAL,PLAN_REVENUE,CICLO,CONSULTOR_DK,CLIENTE_CIUDAD,CLIENTE_REGIONAL,CONSULTOR_CANAL_NOMBRE,CONSULTOR_REGIONAL_NOMBRE,GENERO,MOVIMIENTO_ACTIVACION,JINETE,REGION


In [13]:
ENDING.sample(3)

,SUBSCRIBER_ID,PERIODO_PROCESO_CODIGO,MSISDN,FECHA_ALTA,ANTIGUEDAD,MOVIMIENTO_NOMBRE,ESTADO,MOVIMIENTO_TIPO_NOMBRE,OPERADOR_PORTA_ORIGEN,PLAN_NOMBRE,PLAN_TIPO_COMERCIAL,PLAN_REVENUE,CICLO,CONSULTOR_DK,CLIENTE_CIUDAD,CLIENTE_REGIONAL,CONSULTOR_CANAL_NOMBRE,CONSULTOR_REGIONAL_NOMBRE,GENERO,MOVIMIENTO_ACTIVACION,JINETE,REGION
79802,1169809652,202305,3046473986,2021-08-05 03:16:35,664,PERMANECE,Active,PERMANECE,TIGO,PLAN S,MOBILE,37500,05,1667496,Cartagena,Bolívar,Outbound,Regional Centro,MASCULINO,GROSS ADDS PORTACION,NaN,Regional Costa
468433,1342178611,202305,3244457289,2022-11-28 00:17:20,184,PERMANECE,One-Way Block,PERMANECE,TIGO,PLAN L,MOBILE,45500,01,6341887,Bogotá D.C.,Bogotá D.C.,Outbound,Regional Centro,FEMENINO,GROSS ADDS PORTACION,NaN,Bogotá
822434,2293960647,202305,3026669690,2022-07-30 11:39:16,305,PERMANECE,One-Way Block,PERMANECE,LINEA NUEVA,PLAN S,MOBILE,37500,01,6118730,Medellín,Antioquia,Kiosks,Regional Antioquia,MASCULINO,GROSS ADDS NUEVA,NaN,Regional Antioquia


## *Churnes*

In [14]:
%%time
CHURNERS= pd.read_sql_query(sql=
f"""
WITH 
CHURNERS AS (
    SELECT /*+PARALLEL(6)*/
        CAST(E.SUBSCRIBER_ID AS VARCHAR(20)) SUBSCRIBER_ID,
        CAST(E.MSISDN AS VARCHAR(20)) MSISDN,
        TO_CHAR(E.FECHA_BAJA,'YYYYMMDD') FECHA_BAJA,
        E.MOVIMIENTO_TIPO_NOMBRE AS MOVIMIENT_CHUHRN,
        E.OPERADOR_PORTA_DESTINO,
        ROW_NUMBER() OVER(PARTITION BY E.SUBSCRIBER_ID ORDER BY E.FECHA_ALTA DESC) ORDEN_A
    FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING E
        WHERE SERVICIO='Postpaid'
        AND ESTADO ='Termination'
        AND MOVIMIENTO_TIPO_NOMBRE LIKE '%CHURN%'
        AND PERIODO_PROCESO_CODIGO = {mes + 1}
    )
    
SELECT * FROM CHURNERS
WHERE ORDEN_A = 1
"""
,con=conn)
CHURNERS.shape

Wall time: 18.3 s


(116263, 6)

In [15]:
CHURNERS['MOVIMIENT_CHUHRN'].value_counts()

CHURN PORTACION     57750
CHURN CARTERA       36529
CHURN VOLUNTARIO    15892
CHURN MIGRACION      5526
CHURN FRAUDE          566
Name: MOVIMIENT_CHUHRN, dtype: int64

## *Datos*

In [21]:
%%time
DATOS= pd.read_sql_query(sql=f'''
WITH
DATOS_ULT_30D AS (
SELECT /*PARALLEL(6)*/  
    CAST(SUBSCRIBER_ID AS VARCHAR(20)) SUBSCRIBER_ID,
    SUM(ROUND(BYTES_TOTALES*0.000000001,2)) AS GB_TT_ULT_30D,
    SUM(ROUND((CLARO_3G+TIGO_3G+MOVISTAR_3G+AVANTEL_3G+OTROS_3G)*0.000000001,2)) GB_TT_3G_ULT_30D,
    SUM(ROUND((CLARO_3G+CLARO_4G+TIGO_3G+TIGO_4G+MOVISTAR_3G+MOVISTAR_4G+OTROS_3G)*0.000000001,2)) GB_TT_RAN_ULT_30D,
    SUM(ROUND((AVANTEL_3G+AVANTEL_4G+WOM)*0.000000001,2)) GB_TT_RP_ULT_30D
FROM AGG_DL_DATA
WHERE SERVICIO='Postpaid'
AND PERIODO_PROCESO_CODIGO >= TO_NUMBER(TO_CHAR(TO_DATE(TO_CHAR({dia}),'YYYY/MM/DD') - 30,'YYYYMMDD'))
AND PERIODO_PROCESO_CODIGO < TO_NUMBER(TO_CHAR(TO_DATE(TO_CHAR({dia}),'YYYY/MM/DD'),'YYYYMMDD'))
GROUP BY SUBSCRIBER_ID
)

SELECT /*PARALLEL(6)*/  
    SUBSCRIBER_ID,
    GB_TT_ULT_30D,
    GB_TT_3G_ULT_30D,
    GB_TT_RAN_ULT_30D,
    GB_TT_RP_ULT_30D
FROM DATOS_ULT_30D 
'''
,con=conn)
DATOS.shape


Wall time: 13min 6s


(1506885, 5)

In [17]:
%%time
DATOS_H= pd.read_sql_query(sql=f'''
WITH
DATOS_H AS (
SELECT 
    CAST(D.SUBSCRIBER_ID AS VARCHAR(20)) SUBSCRIBER_ID,
    D.PERIODO,
    ROUND(D.CLARO_3G*0.000000001,2) CLARO_3G, 
    ROUND(D.TIGO_3G*0.000000001,2) TIGO_3G, 
    ROUND(D.MOVISTAR_3G*0.000000001,2) MOVISTAR_3G, 
    ROUND(D.MOVISTAR_4G*0.000000001,2) MOVISTAR_4G, 
    ROUND(D.AVANTEL_3G*0.000000001,2) AVANTEL_3G, 
    ROUND(D.AVANTEL_4G*0.000000001,2) AVANTEL_4G, 
    ROUND(D.WOM*0.000000001,2) WOM, 
    ROUND(D.OTROS_3G*0.000000001,2) OTROS_3G, 
    ROUND(D.BYTES_TOTALES*0.000000001,2) GB_TOTALES
FROM AGG_ML_DATA D
INNER JOIN (
            SELECT /*+PARALLEL(6)*/
                    DISTINCT (SUBSCRIBER_ID)
            FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING 
                WHERE SERVICIO='Postpaid'
                AND ESTADO = 'Active'
                AND PERIODO_PROCESO_CODIGO = {mes}
                AND (FECHA_BAJA IS NULL OR TO_DATE(FECHA_BAJA, 'DD/MM/YY') >= TO_DATE('{today}','DD/MM/YY'))
            ) E ON D.SUBSCRIBER_ID=E.SUBSCRIBER_ID
WHERE D.SERVICIO='Postpaid'
AND TO_NUMBER(D.PERIODO) <= TO_NUMBER(TO_CHAR(ADD_MONTHS(TO_DATE(TO_CHAR({mes}),'YYYY/MM'),-1),'YYYYMM'))
AND TO_NUMBER(D.PERIODO) >= TO_NUMBER(TO_CHAR(ADD_MONTHS(TO_DATE(TO_CHAR({mes}),'YYYY/MM'), -13),'YYYYMM'))
),

DATOS_HIS AS (
SELECT /*PARALLEL(6)*/  
    SUBSCRIBER_ID,
    COUNT(PERIODO) AS MESES,
    SUM(GB_TOTALES) AS GB_TT,
    SUM(CLARO_3G + TIGO_3G + MOVISTAR_3G + AVANTEL_3G + OTROS_3G) AS GB_TT_3G,
    SUM(WOM + AVANTEL_4G + AVANTEL_3G) AS GB_RED_PROPIA,
    SUM (AVANTEL_4G + WOM) /SUM (GB_TOTALES) AS PORC_WOM_4G,
    SUM (AVANTEL_3G) /SUM (GB_TOTALES) AS PORC_WOM_3G,
    SUM (CLARO_3G)/SUM (GB_TOTALES) AS PORC_CLARO_3G , 
    SUM (TIGO_3G)/SUM (GB_TOTALES) AS PORC_TIGO_3G, 
    SUM (MOVISTAR_3G)/SUM (GB_TOTALES) AS PORC_MOVISTAR_3G, 
    SUM (MOVISTAR_4G)/SUM (GB_TOTALES) AS PORC_MOVISTAR_4G,
    SUM (OTROS_3G)/SUM (GB_TOTALES) AS PORC_OTROS_3G
FROM DATOS_H 
WHERE GB_TOTALES > 0
GROUP BY SUBSCRIBER_ID
),

DATOS_HIST AS (
SELECT /*PARALLEL(6)*/
    SUBSCRIBER_ID,
    GB_TT/MESES AS PROM_GB_TT,
    GB_TT_3G/MESES AS PROM_GB_TT_3G,
    GB_RED_PROPIA/MESES AS PROM_GB_RED_PROPIA,
    CASE 
        WHEN MAX_OPERADOR >= 0.2 AND MAX_OPERADOR=PORC_WOM_3G THEN 'WOM_3G'
        WHEN MAX_OPERADOR >= 0.2 AND MAX_OPERADOR=PORC_MOVISTAR_4G THEN 'MOVISTAR_4G'
        WHEN MAX_OPERADOR >= 0.2 AND MAX_OPERADOR=PORC_MOVISTAR_3G THEN 'MOVISTAR_3G'
        WHEN MAX_OPERADOR >= 0.2 AND MAX_OPERADOR=PORC_CLARO_3G THEN 'CLARO_3G'
        WHEN MAX_OPERADOR >= 0.2 AND MAX_OPERADOR=PORC_TIGO_3G THEN 'TIGO_3G'
        WHEN MAX_OPERADOR >= 0.2 AND MAX_OPERADOR=PORC_OTROS_3G THEN 'OTROS_3G'
        WHEN MAX_OPERADOR IS NULL THEN 'SIN TRAFICO'
        ELSE 'WOM_4G' END RED_PRINCIPAL
FROM (
    SELECT /*PARALLEL(6)*/  
        A.*,
        GREATEST(PORC_CLARO_3G, PORC_TIGO_3G, PORC_MOVISTAR_3G, PORC_MOVISTAR_4G,PORC_OTROS_3G,PORC_WOM_3G) MAX_OPERADOR
    FROM DATOS_HIS A)
)


SELECT /*PARALLEL(6)*/  
    SUBSCRIBER_ID,
    PROM_GB_TT,
    PROM_GB_TT_3G,
    PROM_GB_RED_PROPIA,
    RED_PRINCIPAL
FROM DATOS_HIST 
'''
,con=conn)
DATOS_H.shape

Wall time: 3min 16s


(1129674, 5)

## *Minutos*

In [18]:
%%time
MINUTOS_SAL_H= pd.read_sql_query(sql=f'''
WITH 
MIN_SALIENTES AS (
SELECT /*PARALLEL(6)*/  
CAST(V.SUBSCRIBER_ID AS VARCHAR(20)) SUBSCRIBER_ID,
COUNT(V.MES) AS MESES_SAL,
SUM(V.CANT_CALLS) CANT_CALLS_SAL,
SUM(V.DURACION_CALLS) MINUTOS_CALLS_SAL
FROM AGG_ML_VOICE V
INNER JOIN (
            SELECT /*+PARALLEL(6)*/
                    DISTINCT (SUBSCRIBER_ID)
            FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING 
                WHERE SERVICIO='Postpaid'
                AND ESTADO = 'Active'
                AND PERIODO_PROCESO_CODIGO = {mes}
                AND (FECHA_BAJA IS NULL OR TO_DATE(FECHA_BAJA, 'DD/MM/YY') >= TO_DATE('{today}','DD/MM/YY'))
            ) E ON V.SUBSCRIBER_ID=E.SUBSCRIBER_ID
WHERE V.SENTIDO = 'SALIENTE'
AND TO_NUMBER(V.MES) <= TO_NUMBER(TO_CHAR(ADD_MONTHS(TO_DATE(TO_CHAR({mes}),'YYYY/MM'), -1),'YYYYMM'))
AND TO_NUMBER(V.MES) >= TO_NUMBER(TO_CHAR(ADD_MONTHS(TO_DATE(TO_CHAR({mes}),'YYYY/MM'), -13),'YYYYMM'))
GROUP BY V.SUBSCRIBER_ID 
)
SELECT /*PARALLEL(6)*/  
     B.SUBSCRIBER_ID,
    (B.MINUTOS_CALLS_SAL/B.MESES_SAL) PROM_MIN_SAL_MES,
    (B.MINUTOS_CALLS_SAL/B.CANT_CALLS_SAL) PROM_CALLS_SAL
   
FROM MIN_SALIENTES B 
'''
,con=conn)

MINUTOS_SAL_H.shape

Wall time: 4min 16s


(1146646, 3)

In [19]:
%%time
MINUTOS_ENT_H= pd.read_sql_query(sql=f'''
WITH 
MIN_ENTRANTES AS (
SELECT /*PARALLEL(6)*/  
CAST(V.SUBSCRIBER_ID AS VARCHAR(20)) SUBSCRIBER_ID,
COUNT(V.MES) AS MESES_ENT,
SUM(V.CANT_CALLS) CANT_CALLS_ENT,
SUM(V.DURACION_CALLS) MINUTOS_CALLS_ENT
FROM AGG_ML_VOICE V
INNER JOIN (
            SELECT /*+PARALLEL(6)*/
                    DISTINCT (SUBSCRIBER_ID)
            FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING 
                WHERE SERVICIO='Postpaid'
                AND ESTADO = 'Active'
                AND PERIODO_PROCESO_CODIGO = {mes}
                AND (FECHA_BAJA IS NULL OR TO_DATE(FECHA_BAJA, 'DD/MM/YY') >= TO_DATE('{today}','DD/MM/YY'))
            ) E ON V.SUBSCRIBER_ID=E.SUBSCRIBER_ID
WHERE V.SENTIDO = 'ENTRANTE'
AND TO_NUMBER(V.MES) <= TO_NUMBER(TO_CHAR(ADD_MONTHS(TO_DATE(TO_CHAR({mes}),'YYYY/MM'), -1),'YYYYMM'))
AND TO_NUMBER(V.MES) >= TO_NUMBER(TO_CHAR(ADD_MONTHS(TO_DATE(TO_CHAR({mes}),'YYYY/MM'), -13),'YYYYMM'))
GROUP BY V.SUBSCRIBER_ID 
)

SELECT /*PARALLEL(6)*/  
     C.SUBSCRIBER_ID,   
    (C.MINUTOS_CALLS_ENT/C.MESES_ENT) PROM_MIN_ENT_MES,
    (C.MINUTOS_CALLS_ENT/C.CANT_CALLS_ENT) PROM_CALLS_ENT
   
FROM MIN_ENTRANTES C
'''
,con=conn)

MINUTOS_ENT_H.shape

Wall time: 3min 31s


(1131940, 3)

In [20]:
%%time
MINUTOS= pd.read_sql_query(sql=f'''
SELECT /*PARALLEL(6)*/  
    CAST(SUBSCRIBER_ID AS VARCHAR(20)) SUBSCRIBER_ID,
    SUM(CASE WHEN MIN_TOTALES_SALIENTES > 0 THEN 1 ELSE 0 END) CALLS_SAL_ULT30D,
    SUM(ROUND(MIN_TOTALES_SALIENTES,2)) AS MIN_SALIENTES_ULT_30D,
    SUM(CASE WHEN MIN_TOTALES_ENTRANTE > 0 THEN 1 ELSE 0 END) CALLS_ENT_ULT30D,
    SUM(ROUND(MIN_TOTALES_ENTRANTE,2)) AS MIN_ENTRANTES_ULT_30D
FROM CDR_BODEGA_WOM.AGR_TRAFICO_MOU_DIARIO
WHERE SERVICIO='Postpaid'
AND PERIODO_MES > = {mes - 1}
AND PERIODO_DIA >= TO_NUMBER(TO_CHAR(TO_DATE(TO_CHAR({dia}),'YYYY/MM/DD') - 30,'YYYYMMDD'))
AND PERIODO_DIA < TO_NUMBER(TO_CHAR(TO_DATE(TO_CHAR({dia}),'YYYY/MM/DD'),'YYYYMMDD'))
GROUP BY SUBSCRIBER_ID
'''
,con=conn)

MINUTOS.shape

Wall time: 17min 49s


(1657037, 5)

## *Llamadas de la Competencia*

In [22]:
%%time
COMPETENCIA_LLAM= pd.read_sql_query(sql= 
f'''
SELECT /*+PARALLEL(6)*/ 
    CAST(SUBSCRIBER_ID AS VARCHAR(20)) SUBSCRIBER_ID,
    COUNT(*) CANT_LLAMADAS,
    SUM(ROUND((DURACION_SEG/60),2)) MIN_TOTALES,
    ROUND(AVG((DURACION_SEG/60)),2) MIN_PROMEDIO
FROM CDR_BODEGA_WOM.AGR_TRAFICO_MOU_COMPETENCIA
    WHERE SERVICIO='Postpaid'
    AND DURACION_SEG >= 120
    AND OPERADOR !='WOM'
    AND PERIODO_PROCESO_CODIGO >= TO_NUMBER(TO_CHAR(TO_DATE(TO_CHAR({dia}),'YYYY/MM/DD') - 30,'YYYYMMDD'))
    AND PERIODO_PROCESO_CODIGO < TO_NUMBER(TO_CHAR(TO_DATE(TO_CHAR({dia}),'YYYY/MM/DD'),'YYYYMMDD'))
GROUP BY SUBSCRIBER_ID
'''
,con=conn)
COMPETENCIA_LLAM.shape

Wall time: 11.8 s


(177372, 4)

## *SMS*

In [23]:
%%time
SMS= pd.read_sql_query(sql= 
f'''
SELECT /*+PARALLEL(6)*/ 
    CAST(NUM_DESTINO AS VARCHAR(20)) NUM_DESTINO,
    SUM(CASE WHEN TOPICOS = 'OFERTA' THEN 1 ELSE 0 END) CANT_SMS_OFERTAS,
    COUNT(*) CANT_SMS_TT
FROM DP_TRAFICO_SMS
    WHERE FECHA_CARGUE <= {dia}
    AND FECHA_CARGUE >= TO_NUMBER(TO_CHAR(TO_DATE(TO_CHAR({dia}),'YYYY/MM/DD') - 30,'YYYYMMDD'))
GROUP BY NUM_DESTINO
'''
,con=conn)
SMS.shape

Wall time: 57.5 s


(1521818, 3)

## *Cliente llama a la Competencia*

In [24]:
%%time
cliente_llama_competencia= pd.read_sql_query(sql=
f"""
SELECT /*+PARALLEL(6)*/ 
    * 
FROM(
    SELECT /*+PARALLEL(6)*/
        CAST(SUBSCRIBER_ID AS VARCHAR(20)) SUBSCRIBER_ID, 
        FECHA_LLAMADA, 
        LLAMO_A_LA_COMPETENCIA,
        ROW_NUMBER() OVER(PARTITION BY SUBSCRIBER_ID ORDER BY FECHA_LLAMADA DESC) AS ORDEN
    FROM LLAMARON_A_COMPETENCIA
    WHERE SERVICIO = 'Postpaid'
    AND FECHA_LLAMADA < TO_DATE('{today}','dd/mm/yy')
    AND FECHA_LLAMADA >= (TO_DATE('{today}','dd/mm/yy')-30)
)
WHERE ORDEN = 1
"""
,con=conn)

Wall time: 535 ms


## *Quejas*

In [25]:
%%time
QUEJAS= pd.read_sql_query(sql= 
f'''
SELECT /*+PARALLEL(6)*/ 
    CAST(SUBSCRIBER_ID AS VARCHAR(20)) SUBSCRIBER_ID,
    COUNT(*) QUEJAS
FROM CR_DL_PQR
    WHERE SERVICIO='Postpaid'
    AND SERVICE_TYPE2<>'PORTABILIDAD'
    AND TO_NUMBER(TO_CHAR(TO_DATE(CREATED_DATE,'DD/MM/YY'),'YYYYMMDD')) >= TO_NUMBER(TO_CHAR(TO_DATE(TO_CHAR({dia}),'YYYY/MM/DD') - 30,'YYYYMMDD'))
    AND TO_NUMBER(TO_CHAR(TO_DATE(CREATED_DATE,'DD/MM/YY'),'YYYYMMDD')) < TO_NUMBER(TO_CHAR(TO_DATE(TO_CHAR({dia}),'YYYY/MM/DD'),'YYYYMMDD'))
GROUP BY SUBSCRIBER_ID
'''
,con=conn)
QUEJAS.shape

Wall time: 46 s


(281689, 2)

## *PQR*

In [26]:
%%time
PQR= pd.read_sql_query(sql= 
f'''
SELECT /*+PARALLEL(6)*/ 
    CAST(SUBSCRIBER_ID AS VARCHAR(20)) SUBSCRIBER_ID,
    COUNT(*) PQR
FROM CR_DL_PQR
    WHERE SERVICIO='Postpaid'
    AND SERVICE_TYPE2 IN ('LLAMADA TRANSFERIDA A OTRA ÁREA')
    AND TO_NUMBER(TO_CHAR(TO_DATE(CREATED_DATE,'DD/MM/YY'),'YYYYMMDD')) >= TO_NUMBER(TO_CHAR(TO_DATE(TO_CHAR({dia}),'YYYY/MM/DD') - 30,'YYYYMMDD'))
    AND TO_NUMBER(TO_CHAR(TO_DATE(CREATED_DATE,'DD/MM/YY'),'YYYYMMDD')) < TO_NUMBER(TO_CHAR(TO_DATE(TO_CHAR({dia}),'YYYY/MM/DD'),'YYYYMMDD'))
GROUP BY SUBSCRIBER_ID
'''
,con=conn)
PQR.shape

Wall time: 14.4 s


(61705, 2)

## *Recurrencia*

In [27]:
%%time
Recurrencia= pd.read_sql_query(sql= 
f'''
SELECT /*+ PARALLEL(6) */ 
    CAST(SUBSCRIBER_ID AS VARCHAR(20)) SUBSCRIBER_ID,
    CASE WHEN (CANT_PETICION>1 OR CANT_QUEJA>1 OR CANT_CONSULTAS> 1) THEN 'REINCIDENTE'
        WHEN (CANT_PETICION+CANT_QUEJA+CANT_CONSULTAS)>1 THEN 'RECURRENTE'
        ELSE 'PQRS_UNICO' END AS GRUPO_REC
FROM (
    SELECT /*+ PARALLEL(6) */ 
        SUBSCRIBER_ID,
        SUM(CASE WHEN SERVICE_TYPE1 = 'CONSULTAS' THEN 1 ELSE 0 END) CANT_CONSULTAS,
        SUM(CASE WHEN SERVICE_TYPE1 = 'PETICIÓN' THEN 1 ELSE 0 END) CANT_PETICION,
        SUM(CASE WHEN SERVICE_TYPE1 = 'QUEJA' THEN 1 ELSE 0 END) CANT_QUEJA
    FROM CR_DL_PQR
        WHERE SERVICIO='Postpaid'
        AND SERVICE_TYPE1 IN ('QUEJA','PETICIÓN','CONSULTAS')
        AND TO_NUMBER(TO_CHAR(TO_DATE(CREATED_DATE,'DD/MM/YY'),'YYYYMMDD')) >= TO_NUMBER(TO_CHAR(TO_DATE(TO_CHAR({dia}),'YYYY/MM/DD') - 90,'YYYYMMDD'))
        AND TO_NUMBER(TO_CHAR(TO_DATE(CREATED_DATE,'DD/MM/YY'),'YYYYMMDD')) < TO_NUMBER(TO_CHAR(TO_DATE(TO_CHAR({dia}),'YYYY/MM/DD'),'YYYYMMDD'))
    GROUP BY SUBSCRIBER_ID)
'''
,con=conn)
Recurrencia.shape

Wall time: 1min 40s


(546756, 2)

## *Cantidad de Lineas*

In [28]:
%%time
CANT_LINEAS= pd.read_sql_query(sql= f""" 
SELECT /*+ PARALLEL(6) */  
    CAST(SUBSCRIBER_ID AS VARCHAR(20)) SUBSCRIBER_ID,
    ROW_NUMBER() OVER(PARTITION BY CLIENTE_DK,PERIODO_PROCESO_CODIGO ORDER BY FECHA_ALTA ,MOVIMIENTO_NOMBRE DESC) AS NUM_LINEA_CLI
FROM (
    SELECT /*+ PARALLEL(6) */ 
        SUBSCRIBER_ID, 
        MOVIMIENTO_NOMBRE, 
        MOVIMIENTO_TIPO_NOMBRE, 
        PERIODO_PROCESO_CODIGO,
        FECHA_ALTA,FECHA_BAJA,
        CLIENTE_DK,
        ROW_NUMBER() OVER(PARTITION BY SUBSCRIBER_ID,PERIODO_PROCESO_CODIGO ORDER BY PERIODO_PROCESO_CODIGO DESC,MOVIMIENTO_NOMBRE DESC) AS ORDEN
    FROM  DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING
        WHERE SERVICIO='Postpaid'
        AND PERIODO_PROCESO_CODIGO ={mes}
        ORDER BY SUBSCRIBER_ID, ORDEN DESC) A
WHERE A.ORDEN=1
AND PERIODO_PROCESO_CODIGO={mes}
AND CLIENTE_DK NOT IN (-1,-2)
"""
,con=conn)
CANT_LINEAS.shape

Wall time: 19.1 s


(1701364, 2)

## *Facturación*

In [29]:
%%time
FACTURA= pd.read_sql_query(sql= f""" 
WITH 
ULT_FAC AS (
SELECT /*+PARALLEL(4)*/  
    A.*
    ,ROW_NUMBER() OVER(PARTITION BY A.SUBS_ID ORDER BY A.FECHA_FACTURACION DESC) ORDEN
FROM CR_DL_PAGOS A
WHERE TO_NUMBER(TO_CHAR(A.FECHA_FACTURACION,'YYYYMMDD')) <= {dia}
)

SELECT /*+ PARALLEL(4) */  
    CAST(B.SUBS_ID AS VARCHAR(20)) SUBSCRIBER_ID,
    C.DEUDA AS ULT_FAC, 
    AVG(B.DEUDA) AS PROM_FACTURA,
    SUM(B.DEUDA) AS VAL_FACTURADO,
    SUM(B.PAGO)*(-1) AS VALOR_PAGADO,
    SUM(CASE WHEN B.DEUDA > 0 THEN 1 ELSE 0 END) AS CANT_FACTURAS,
    SUM(CASE WHEN B.PAGO*(-1) > 0 THEN 1 ELSE 0 END) AS CANT_PAGOS,
    SUM(CASE WHEN B.DIAS_PAGO_VENCIMIENTO <= 0 THEN 1 ELSE 0 END) CANT_PAGOS_TIEMPO,
    SUM(CASE WHEN B.DIAS_PAGO_VENCIMIENTO > 0 THEN 1 ELSE 0 END) CANT_PAGOS_MORA
FROM CR_DL_PAGOS B
LEFT JOIN ULT_FAC C ON (B.SUBS_ID=C.SUBS_ID AND C.ORDEN=1)
WHERE TO_NUMBER(TO_CHAR(B.FECHA_FACTURACION,'YYYYMMDD')) <= {dia}
GROUP BY B.SUBS_ID,C.DEUDA
"""
,con=conn)

FACTURA.shape

Wall time: 55min 31s


(3170115, 9)

## *Gross Churn*

In [30]:
%%time
Gross_churn= pd.read_sql_query(sql=
f""" 
SELECT /*+PARALLEL(6)*/
    CAST(SUBSCRIBER_ID AS VARCHAR(20)) SUBSCRIBER_ID,
    PERIODO_PROCESO_CODIGO, 
    ALTA_PROB
FROM ANALITICA_WOM.MODELO_GROSS_CHURN_V2
WHERE PREDICT_PROBA >0.85
AND PERIODO_PROCESO_CODIGO = {mes}
"""
,con=conn)

Gross_churn.shape

Wall time: 1.69 s


(9084, 3)

## *Compra de Equipos*

In [31]:
%%time
Compr_Equi= pd.read_sql_query(sql=
f""" 
SELECT /*+PARALLEL(6)*/
    SUBSCRIBER_ID,
    COMPRO_EQUIPO
FROM (
SELECT /*+PARALLEL(6)*/
    CAST(SUBSCRIBER_ID AS VARCHAR(20)) SUBSCRIBER_ID,
    1 COMPRO_EQUIPO,
    ROW_NUMBER() OVER(PARTITION BY SUBSCRIBER_ID ORDER BY PERIODO_PROCESO_CODIGO DESC) ORDEN
FROM  DWH_BODEGA_WOM.FCT_DEVICES
WHERE SERVICIO = 'Postpaid'
AND PERIODO_PROCESO_CODIGO >= TO_NUMBER(TO_CHAR(ADD_MONTHS(TO_DATE(TO_CHAR({mes}),'YYYY/MM'), -6),'YYYYMM'))
AND PERIODO_PROCESO_CODIGO <= TO_NUMBER({mes})
)
WHERE ORDEN = 1
"""
,con=conn)

Compr_Equi.shape

Wall time: 4.09 s


(208153, 2)

## *Buro*

In [32]:
%%time
Buro= pd.read_sql_query(sql=
f""" 
WITH 
BURO AS ( 
SELECT /*+PARALLEL(6)*/
SUBSCRIBER_ID,
CASE 
    WHEN TIPO_BURO = '01 Riesgo Bajo' THEN 'BAJO'
    WHEN TIPO_BURO = '02 Riesgo Medio' THEN 'MEDIO'
    WHEN TIPO_BURO = '03 Riesgo Medio-Alto' THEN 'MEDIO-ALTO'
    WHEN TIPO_BURO = 'Sin Buro' THEN 'SIN RIESGO'
    WHEN TIPO_BURO LIKE '%Riesgo Alto%' THEN 'ALTO'
    ELSE 'NA' END RIESGO,   
    
ROW_NUMBER() OVER(PARTITION BY SUBSCRIBER_ID ORDER BY PERIODO_PROCESO_CODIGO DESC) ORDEN
FROM BI_WOM.TBL_CONSOLIDADO_GROSS_SCORE
WHERE SERVICIO = 'Postpaid'
)

SELECT /*+PARALLEL(6)*/ 
CAST(SUBSCRIBER_ID AS VARCHAR(20)) SUBSCRIBER_ID,
RIESGO
FROM BURO
WHERE ORDEN = 1 
"""
,con=conn)

Buro.shape

Wall time: 20.3 s


(1733468, 2)

## *Segunda Linea*

In [33]:
%%time
Segunda_Linea= pd.read_sql_query(sql=
f""" 
WITH
SEGUNDA_LINEA AS (
    SELECT /*+PARALLEL(6)*/
        SUBSCRIBER_ID,
        TIPO,
        PERIODO_OFERTA,
        SUBS_CON_DESCUENTO
    FROM  (
        SELECT /*+PARALLEL(6)*/
            SUBS_A SUBSCRIBER_ID,
            TIPO,
            PERIODO_OFERTA,
            SUBS_CON_DESCUENTO,
            ACTIVO
        FROM COMERCIAL_WOM.PROGRAMACIONES_DESCUENTO_2X1
        UNION ALL
        SELECT /*+PARALLEL(6)*/
            SUBS_B SUBSCRIBER_ID,
            TIPO,
            PERIODO_OFERTA,
            SUBS_CON_DESCUENTO,
            ACTIVO
        FROM COMERCIAL_WOM.PROGRAMACIONES_DESCUENTO_2X1 )
        GROUP BY
            SUBSCRIBER_ID,
            TIPO,
            PERIODO_OFERTA,
            SUBS_CON_DESCUENTO,
            ACTIVO
)

SELECT /*+PARALLEL(6)*/
SUBSCRIBER_ID,
SEGUNDA_LINEA
FROM (
SELECT /*+PARALLEL(6)*/
    CAST(A.SUBSCRIBER_ID AS VARCHAR(20)) SUBSCRIBER_ID,
    1 AS SEGUNDA_LINEA,
    ROW_NUMBER() OVER(PARTITION BY SUBSCRIBER_ID ORDER BY PERIODO_OFERTA DESC) ORDEN
FROM SEGUNDA_LINEA  A)
WHERE ORDEN = 1
"""
,con=conn)

Segunda_Linea.shape

Wall time: 12 s


(758538, 2)

## *Codigos*

In [ ]:
%%time
Codigos= pd.read_sql_query(sql=
f""" 
WITH 
CODIGO AS (
SELECT /*+PARALLEL(6)*/
TELEFONO AS MSISDN,
TO_DATE(FECHA_ENVIO,'DD/MM/YYYY') FECHA_ENVIO,
1 NIP
FROM DP_GESTION_MOV_PRIORIZADA
WHERE TO_NUMBER(TO_CHAR(FECHA_ENVIO,'YYYYMMDD')) >= TO_NUMBER(TO_CHAR(TO_DATE(TO_CHAR({dia}),'YYYY/MM/DD') - 60,'YYYYMMDD'))
AND TO_NUMBER(TO_CHAR(FECHA_ENVIO,'YYYYMMDD')) <= TO_NUMBER(TO_CHAR(TO_DATE(TO_CHAR({dia}),'YYYY/MM/DD'),'YYYYMMDD'))
),

CODIGO1 AS (
SELECT  /*+PARALLEL(6)*/
CAST(C.MSISDN AS VARCHAR(20)) MSISDN,
C.FECHA_ENVIO,
C.NIP,
A.CREATED_DATE,
A.EFF_DATE,
CASE 
    WHEN A.CREATED_DATE IS NULL THEN 0 
    WHEN A.CREATED_DATE > C.FECHA_ENVIO  THEN 0 
    ELSE 1 END PORTACION_CREADA,
(TO_DATE(CREATED_DATE,'DD/MM/YYYY') - TO_DATE(FECHA_ENVIO,'DD/MM/YYYY')) AS DIFF_RESIVE_CREA,
ROW_NUMBER() OVER(PARTITION BY MSISDN ORDER BY FECHA_ENVIO DESC) ORDEN
    
FROM CODIGO C
LEFT JOIN (
    SELECT /*+PARALLEL(6)*/
        PORT_NUMBER,
        TO_DATE(CREATED_DATE,'DD/MM/YYYY') AS CREATED_DATE,
        TO_DATE(EFF_DATE,'DD/MM/YYYY') AS EFF_DATE
    FROM COMERCIAL_WOM.PORT_OUT_WOM 
    WHERE PERIODO_PROCESO_CODIGO >= {mes-1}
    AND  PERIODO_PROCESO_CODIGO <= {mes}
    AND NP_ORDER_NAME IN ('Complete','Waiting broadcast','Order Scheduling')
    ) A ON C.MSISDN = A.PORT_NUMBER
)

SELECT /*+PARALLEL(6)*/
    MSISDN,
    NIP,
    PORTACION_CREADA,
    DIFF_RESIVE_CREA
FROM CODIGO1
WHERE ORDEN = 1
"""
,con=conn)
Codigos.shape

## *Descuentos*

In [ ]:
%%time
Descuentos= pd.read_sql_query(sql=
f""" 
WITH 
FACTU AS (
SELECT /*+PARALLEL(6)*/
    A.PERIODO_PROCESO_CODIGO, 
    A.SUBSCRIBER_ID, 
    CASE WHEN A.PLAN_REVENUE-(B.DEUDA-B.LDI) < 0 THEN (B.DEUDA-B.LDI) ELSE A.PLAN_REVENUE END PLAN_REVENUE,
    B.DEUDA-B.LDI DEUDA,
    CASE WHEN A.PLAN_REVENUE-(B.DEUDA-B.LDI) < 0 THEN 0 ELSE A.PLAN_REVENUE-(B.DEUDA-B.LDI) END DESCUENTO_DEUDA,
    ROW_NUMBER() OVER(PARTITION BY A.SUBSCRIBER_ID ORDER BY A.PERIODO_PROCESO_CODIGO DESC) AS ORDEN
FROM ENDING_ARPU A
LEFT JOIN ANALITICA_WOM.TEMP_FACT_PR B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID AND A.PERIODO_PROCESO_CODIGO=B.PERIODO_PROCESO_CODIGO
WHERE A. PERIODO_PROCESO_CODIGO = {mes}
)

SELECT /*+PARALLEL(6)*/
PERIODO_PROCESO_CODIGO,
CAST(SUBSCRIBER_ID AS VARCHAR(20)) SUBSCRIBER_ID,
CASE WHEN DESCUENTO_DEUDA IS NOT NULL AND DESCUENTO_DEUDA > 0 THEN 1 ELSE 0 END DESCUENTO 
FROM FACTU 
WHERE ORDEN = 1
"""
,con=conn)
Descuentos.shape

## *Une Insumos*

In [ ]:
ENDING.shape

In [ ]:
%%time
BD = pd.merge(ENDING, CHURNERS[['SUBSCRIBER_ID','FECHA_BAJA','MOVIMIENT_CHUHRN','OPERADOR_PORTA_DESTINO']], how='left', left_on=['SUBSCRIBER_ID'], right_on=['SUBSCRIBER_ID'])
BD = pd.merge(BD, DATOS[['SUBSCRIBER_ID', 'GB_TT_ULT_30D', 'GB_TT_3G_ULT_30D','GB_TT_RAN_ULT_30D','GB_TT_RP_ULT_30D']], how='left', left_on=['SUBSCRIBER_ID'], right_on=['SUBSCRIBER_ID'])
BD = pd.merge(BD, DATOS_H[['SUBSCRIBER_ID', 'PROM_GB_TT', 'PROM_GB_TT_3G', 'PROM_GB_RED_PROPIA','RED_PRINCIPAL']], how='left', left_on=['SUBSCRIBER_ID'], right_on=['SUBSCRIBER_ID'])
BD = pd.merge(BD, MINUTOS, how='left', left_on=['SUBSCRIBER_ID'], right_on=['SUBSCRIBER_ID'])
BD = pd.merge(BD, MINUTOS_ENT_H, how='left', left_on=['SUBSCRIBER_ID'], right_on=['SUBSCRIBER_ID'])
BD = pd.merge(BD, MINUTOS_SAL_H, how='left', left_on=['SUBSCRIBER_ID'], right_on=['SUBSCRIBER_ID'])
BD = pd.merge(BD, COMPETENCIA_LLAM, how='left', left_on=['SUBSCRIBER_ID'], right_on=['SUBSCRIBER_ID'])
BD = pd.merge(BD, SMS, how='left', left_on=['MSISDN'], right_on=['NUM_DESTINO'])
BD = pd.merge(BD, cliente_llama_competencia[['SUBSCRIBER_ID','LLAMO_A_LA_COMPETENCIA']], how='left', left_on=['SUBSCRIBER_ID'], right_on=['SUBSCRIBER_ID'])
BD = pd.merge(BD, QUEJAS, how='left', left_on=['SUBSCRIBER_ID'], right_on=['SUBSCRIBER_ID'])
BD = pd.merge(BD, PQR, how='left', left_on=['SUBSCRIBER_ID'], right_on=['SUBSCRIBER_ID'])
BD = pd.merge(BD, Recurrencia, how='left', left_on=['SUBSCRIBER_ID'], right_on=['SUBSCRIBER_ID'])
BD = pd.merge(BD, CANT_LINEAS, how='left', left_on=['SUBSCRIBER_ID'], right_on=['SUBSCRIBER_ID'])
BD = pd.merge(BD, FACTURA, how='left', left_on=['SUBSCRIBER_ID'], right_on=['SUBSCRIBER_ID'])
BD = pd.merge(BD, Gross_churn[['SUBSCRIBER_ID','ALTA_PROB']], how='left', left_on=['SUBSCRIBER_ID'], right_on=['SUBSCRIBER_ID'])
BD = pd.merge(BD, Compr_Equi, how='left', left_on=['SUBSCRIBER_ID'], right_on=['SUBSCRIBER_ID'])
BD = pd.merge(BD, Buro, how='left', left_on=['SUBSCRIBER_ID'], right_on=['SUBSCRIBER_ID'])
BD = pd.merge(BD, Segunda_Linea[['SUBSCRIBER_ID','SEGUNDA_LINEA']], how='left', left_on=['SUBSCRIBER_ID'], right_on=['SUBSCRIBER_ID'])
BD = pd.merge(BD, Codigos[['MSISDN','NIP','PORTACION_CREADA','DIFF_RESIVE_CREA']], how='left', left_on=['MSISDN'], right_on=['MSISDN'])
BD = pd.merge(BD, Descuentos[['SUBSCRIBER_ID','DESCUENTO']], how='left', left_on=['SUBSCRIBER_ID'], right_on=['SUBSCRIBER_ID'])

BD.shape

## *Mineria de Datos*

In [ ]:
BD['PLAN_REVENUE']=BD['PLAN_REVENUE'].fillna(0)
BD['PLAN_REVENUE']=BD['PLAN_REVENUE'].astype(int)

In [ ]:
BD['CICLO']=BD['CICLO'].fillna(0)
BD['CICLO']=BD['CICLO'].astype(int)

In [ ]:
BD['CHURN_PORTACION']= np.where(BD['MOVIMIENT_CHUHRN']=='CHURN PORTACION', 1,0 )
BD['CHURN']= np.where(BD['MOVIMIENT_CHUHRN'].isna() , 0,1)

In [ ]:
BD['CHURN_PORT_15D']= np.where(BD['FECHA_BAJA']<='20230515', 1,0 )

In [ ]:
BD['CHURN2']= np.where(BD['FECHA_BAJA'].isna() , 0,1)

In [ ]:
BD['CHURN'].value_counts().sort_values()

In [ ]:
BD['CHURN2'].value_counts().sort_values()

In [ ]:
BD['CHURN_PORTACION'].value_counts().sort_values()

In [ ]:
BD.head(5)

In [ ]:
BD_Numericas = BD[['SUBSCRIBER_ID', 'MSISDN', 'ANTIGUEDAD', 'PLAN_REVENUE', 'CICLO', 'JINETE', 
                   'GB_TT_ULT_30D', 'GB_TT_3G_ULT_30D','GB_TT_RAN_ULT_30D','GB_TT_RP_ULT_30D', 'PROM_GB_TT', 'PROM_GB_TT_3G','PROM_GB_RED_PROPIA', 
                   'PROM_MIN_SAL_MES', 'PROM_CALLS_SAL','PROM_MIN_ENT_MES', 'PROM_CALLS_ENT', 'MIN_SALIENTES_ULT_30D','CALLS_SAL_ULT30D', 
                   'MIN_ENTRANTES_ULT_30D', 'CALLS_ENT_ULT30D','CANT_LLAMADAS', 'MIN_TOTALES', 'MIN_PROMEDIO', 'CANT_SMS_OFERTAS','CANT_SMS_TT', 
                   'LLAMO_A_LA_COMPETENCIA', 'QUEJAS', 'PQR','NUM_LINEA_CLI', 'ULT_FAC', 'PROM_FACTURA', 'VAL_FACTURADO','VALOR_PAGADO', 
                   'CANT_FACTURAS', 'CANT_PAGOS', 'CANT_PAGOS_TIEMPO','CANT_PAGOS_MORA', 'ALTA_PROB', 'COMPRO_EQUIPO', 'SEGUNDA_LINEA', 'NIP',
                   'PORTACION_CREADA', 'DIFF_RESIVE_CREA', 'DESCUENTO']]
BD_Numericas=BD_Numericas.fillna(0)
BD_Numericas.shape

In [ ]:
BD_Numericas=BD_Numericas.fillna(0)

In [ ]:
BD_Categoricas = BD[['SUBSCRIBER_ID', 'MSISDN', 'MOVIMIENTO_ACTIVACION', 'OPERADOR_PORTA_ORIGEN', 'PLAN_NOMBRE','PLAN_TIPO_COMERCIAL','CONSULTOR_CANAL_NOMBRE', 'GENERO', 
                     'REGION','MOVIMIENT_CHUHRN', 'RED_PRINCIPAL', 'GRUPO_REC', 'RIESGO','FECHA_ALTA' ,'FECHA_BAJA']]
BD_Categoricas.shape

In [ ]:
BD_Categoricas=BD_Categoricas.fillna('REV')

In [ ]:
BD_Categoricas['OPERADOR_PORTA_ORIGEN']= np.where(BD_Categoricas['MOVIMIENTO_ACTIVACION'] == 'GROSS ADDS NUEVA', 'LINEA NUEVA',
                                                  np.where(BD_Categoricas['MOVIMIENTO_ACTIVACION'] == 'GROSS ADDS MIGRACION', 'WOM',
                                                           np.where(((BD_Categoricas['MOVIMIENTO_ACTIVACION'] == 'GROSS ADDS PORTACION') & (BD_Categoricas['OPERADOR_PORTA_ORIGEN']=='LINEA NUEVA')), 'OTROS',
                                                                    BD_Categoricas['OPERADOR_PORTA_ORIGEN'])))

In [ ]:
BD_Categoricas['GENERO']= np.where(BD_Categoricas['GENERO']=='REV', 'SIN CLASIFICAR',BD_Categoricas['GENERO'])
BD_Categoricas['MOVIMIENT_CHUHRN']= np.where(BD_Categoricas['MOVIMIENT_CHUHRN']=='REV', 'PERMANECE',BD_Categoricas['MOVIMIENT_CHUHRN'])
BD_Categoricas['RED_PRINCIPAL']= np.where(BD_Categoricas['RED_PRINCIPAL']=='REV', 'SIN TRAFICO',BD_Categoricas['RED_PRINCIPAL'])
BD_Categoricas['GRUPO_REC']= np.where(BD_Categoricas['GRUPO_REC']=='REV', 'SIN PQRS',BD_Categoricas['GRUPO_REC'])
BD_Categoricas['RIESGO']= np.where(BD_Categoricas['RIESGO']=='REV', 'SIN RIESGO',BD_Categoricas['RIESGO'])

In [ ]:
#BD_Categoricas[['PLAN_TIPO_COMERCIAL','PLAN_NOMBRE']].value_counts().sort_index()
#sort_index()

In [ ]:
BD_Final =  pd.merge(BD_Categoricas, BD_Numericas, how='left', left_on=['SUBSCRIBER_ID','MSISDN'], right_on=['SUBSCRIBER_ID','MSISDN'])
BD_Final.shape

In [ ]:
BD_Final['MESES_ACTIVO'] = np.where (BD_Final['ANTIGUEDAD'] == 0 , 0 ,BD_Final['ANTIGUEDAD'] /30)
BD_Final['MESES_ACTIVO'] = round(BD_Final['MESES_ACTIVO'] , 1) 

In [ ]:
BD_Final['CAMBIO_CONSUMO_GB']=(BD_Final['GB_TT_ULT_30D']-BD_Final['PROM_GB_TT'])/BD_Final['PROM_GB_TT']
BD_Final['DISMINUYO_CONSUMO_GB']=np.where(((BD_Final.CAMBIO_CONSUMO_GB<0)),1,0)

In [ ]:
BD_Final['ATRAIDO_COMPE']=np.where((BD_Final['CANT_SMS_TT']>=1)&(BD_Final['LLAMO_A_LA_COMPETENCIA']>=1),1,0)

In [ ]:
BD_Final['COSTO_GB_ULT_30'] = BD_Final.GB_TT_ULT_30D/(BD_Final.PLAN_REVENUE/30)
BD_Final.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
BD_Final['COSTO_GB_HIST'] = BD_Final.PROM_GB_TT/(BD_Final.PLAN_REVENUE/30)
BD_Final.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
BD_Final['CAMBIO_COSTO_GB'] = (BD_Final['COSTO_GB_ULT_30']-BD_Final['COSTO_GB_HIST'])
BD_Final['IMPORTE_GASTADO_WOM'] = BD_Final.PLAN_REVENUE*BD_Final.MESES_ACTIVO

In [ ]:
BD_Final['COSTO_GB_ULT_30'] = BD_Final['COSTO_GB_ULT_30'].fillna(0)
BD_Final['CAMBIO_COSTO_GB'] = BD_Final['CAMBIO_COSTO_GB'].fillna(0)

In [ ]:
BD_Final.shape

In [ ]:
BD_Final.head(5)

In [ ]:
BD_Final.to_csv('BD_Final.csv', index=False)

In [ ]:
BD_Mod = BD_Final[BD_Final['ALTA_PROB']==0].copy()
BD_Mod = BD_Mod[BD_Mod['PORTACION_CREADA']==0].copy()
BD_Mod.shape

In [ ]:
prueba = pd.DataFrame(BD_Final[['MOVIMIENT_CHUHRN','FECHA_BAJA']].value_counts().sort_index()).reset_index()
prueba.head(120)

In [ ]:
prueba.to_csv('prueba.csv', index=False)

## *Preparo Base Final para Modelo*

In [ ]:
BD_Modelo = BD_Mod[['SUBSCRIBER_ID', 'MSISDN', 'MOVIMIENTO_ACTIVACION','OPERADOR_PORTA_ORIGEN', 'PLAN_NOMBRE', 'PLAN_TIPO_COMERCIAL','CONSULTOR_CANAL_NOMBRE', 'GENERO', 
                      'REGION', 'RED_PRINCIPAL', 'GRUPO_REC', 'RIESGO','PLAN_REVENUE', 'CICLO', 'JINETE', 'GB_TT_ULT_30D','GB_TT_3G_ULT_30D', 'GB_TT_RAN_ULT_30D', 'GB_TT_RP_ULT_30D',
                      'PROM_GB_TT', 'PROM_GB_TT_3G','PROM_GB_RED_PROPIA', 'PROM_MIN_SAL_MES', 'PROM_CALLS_SAL','PROM_MIN_ENT_MES', 'PROM_CALLS_ENT', 'MIN_SALIENTES_ULT_30D',
                      'CALLS_SAL_ULT30D', 'MIN_ENTRANTES_ULT_30D', 'CALLS_ENT_ULT30D','CANT_LLAMADAS', 'MIN_TOTALES', 'MIN_PROMEDIO', 'CANT_SMS_OFERTAS','CANT_SMS_TT', 
                      'LLAMO_A_LA_COMPETENCIA', 'QUEJAS', 'PQR','NUM_LINEA_CLI', 'ULT_FAC', 'PROM_FACTURA', 'VAL_FACTURADO','VALOR_PAGADO', 'CANT_FACTURAS', 'CANT_PAGOS', 
                      'CANT_PAGOS_TIEMPO','CANT_PAGOS_MORA', 'COMPRO_EQUIPO', 'SEGUNDA_LINEA', 'NIP', 'DESCUENTO', 'MESES_ACTIVO','CAMBIO_CONSUMO_GB', 'DISMINUYO_CONSUMO_GB', 
                      'ATRAIDO_COMPE','COSTO_GB_ULT_30', 'COSTO_GB_HIST', 'CAMBIO_COSTO_GB','IMPORTE_GASTADO_WOM', 'CHUNR', 'CHUNR_PORTACION']].copy()



In [ ]:
print(BD_Modelo.shape)

In [ ]:
from feature_engine.encoding import MeanEncoder

In [ ]:
mean_enc= MeanEncoder(variables=['MOVIMIENTO_ACTIVACION','OPERADOR_PORTA_ORIGEN', 'PLAN_NOMBRE', 'PLAN_TIPO_COMERCIAL','CONSULTOR_CANAL_NOMBRE', 'GENERO','REGION', 
                                 'RED_PRINCIPAL', 'GRUPO_REC', 'RIESGO'])
mean_enc.fit(BD_Modelo,BD_Modelo.CHUNR_PORTACION)
BD_Modelo=mean_enc.transform(BD_Modelo)


In [ ]:
BD_Modelo.to_csv('BD_Modelo.csv', index=False)